In [1]:
from keplergl import KeplerGl
import json
import networkx
import uuid

censimento = json.load(open('dataset_completo.geojson'))
    
#Load Map config
%run 'Censimento_Milano_2011/config.py'

map = KeplerGl(data={"censimento_milano_2011": censimento},config=config)
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'k75ulfp', 'type': …

In [2]:
#Sezione-based randomic family generator

settori = []
id_familiare = 1

for i in censimento.get("features"):
    if (i.get("properties").get("POP_2010") == 0 or i.get("indicatori").get("1") == '' or i.get("indicatori").get("1") == "0"):
        continue
    print("Codice sezione:",i.get("properties").get("SEZ"))
    POP_2010 = i.get("properties").get("POP_2010")
    edifici = round(POP_2010/float(i.get("indicatori").get("1")))
    appartamenti = round(edifici*float(i.get("indicatori").get("3")))
    print("Popolazione:",POP_2010)
    print("Numero di edifici ad uso residenziale:", edifici)
    print("Numero di abitazioni totali ad uso residenziale:", appartamenti)
    if (appartamenti!=0):
        print("Numero di abitanti per appartamento ad uso residenziale:", POP_2010/appartamenti)
        print("Numero medio di componenti delle famiglie:", i.get("indicatori").get("39"))
    #print("Percentuale di 65+ che vivono da soli:",i.get("indicatori").get("17"))
    #print("Percentuale di bambini in età prescolare:",i.get("indicatori").get("8"))

    #Percentuali esterne alla fascia 15-64
    if (i.get("indicatori").get("25") != '' and i.get("indicatori").get("26") != ''):
        young = float(i.get("indicatori").get("25"))
        old = float(i.get("indicatori").get("26"))
        Y = old + young

        X = (100+Y)/100
        mid_percent = 100/X
        old_percent = old/X
        young_percent = young/X

        anziani = round((POP_2010/100)*(old_percent))
        giovani = round((POP_2010/100)*(young_percent))
        attivi = round((POP_2010/100)*mid_percent)

        print("Abitanti nella fascia 15-64:", attivi)
        #print("Abitanti fuori dalla fascia 15-64:",anziani+giovani)
        print("Abitanti < 15:", giovani)
        print("Abitanti > 65:", anziani)
        print("Over 65+ che vivono da soli:",round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27")))))
        print()

        settore = []

        #65+ old solo families generator
        for j in range (0,round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))):
            fam = {}
            members = []
            obj = {}
            obj["uuid"] = uuid.uuid4()
            obj["age"] = "old"
            fam["family_id"] = id_familiare
            members.append(obj)
            fam["members"] = members
            id_familiare = id_familiare + 1
            settore.append(fam)

        #Remove created families from 65+
        POP_2010 = POP_2010 - round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))
        appartamenti = appartamenti - round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))

        print()
        print(i.get("indicatori").get("14"))
        if (POP_2010!=round(((POP_2010/100)*mid_percent+(POP_2010/100)*(old_percent + young_percent)))):
            print(POP_2010)
            print(round((POP_2010/100)*mid_percent+(POP_2010/100)*(old_percent + young_percent)))
        if (i.get("properties").get("SEZ")==3):
            break
    


 


Codice sezione: 1
Popolazione: 15.0
Numero di edifici ad uso residenziale: 1
Numero di abitazioni totali ad uso residenziale: 8
Numero di abitanti per appartamento ad uso residenziale: 1.875
Numero medio di componenti delle famiglie: 1.75
Abitanti nella fascia 15-64: 6
Abitanti < 15: 1
Abitanti > 65: 8
Over 65+ che vivono da soli: 2


0
Codice sezione: 2
Popolazione: 21.0
Numero di edifici ad uso residenziale: 2
Numero di abitazioni totali ad uso residenziale: 9
Numero di abitanti per appartamento ad uso residenziale: 2.3333333333333335
Numero medio di componenti delle famiglie: 2.4
Abitanti nella fascia 15-64: 14
Abitanti < 15: 4
Abitanti > 65: 3
Over 65+ che vivono da soli: 0


50
Codice sezione: 3
Popolazione: 22.0
Numero di edifici ad uso residenziale: 2
Numero di abitazioni totali ad uso residenziale: 18
Numero di abitanti per appartamento ad uso residenziale: 1.2222222222222223
Numero medio di componenti delle famiglie: 1.11
Abitanti nella fascia 15-64: 7
Abitanti < 15: 0
Abitant